$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 6: Min-Conflicts

Lucas Wilson

Add more sections to present your code, results, and discussions.*

# Code

## Imports

In [1]:
import random

## Given Code

In [2]:
def min_conflicts(vars, domains, constraints, neighbors, max_steps=1000): 
    """Solve a CSP by stochastic hillclimbing on the number of conflicts."""
    # Generate a complete assignment for all vars (probably with conflicts)
    current = {}
    for var in vars:
        val = min_conflicts_value(var, current, domains, constraints, neighbors)
        current[var] = val
    # Now repeatedly choose a random conflicted variable and change it
    for i in range(max_steps):
        conflicted = conflicted_vars(current,vars,constraints,neighbors)
        if not conflicted:
            return (current,i)
        var = random.choice(conflicted)
        val = min_conflicts_value(var, current, domains, constraints, neighbors)
        current[var] = val
    return (None,None)

def min_conflicts_value(var, current, domains, constraints, neighbors):
    """Return the value that will give var the least number of conflicts.
    If there is a tie, choose at random."""
    return argmin_random_tie(domains[var],
                             lambda val: nconflicts(var, val, current, constraints, neighbors)) 

def conflicted_vars(current,vars,constraints,neighbors):
    "Return a list of variables in current assignment that are in conflict"
    return [var for var in vars
            if nconflicts(var, current[var], current, constraints, neighbors) > 0]

def nconflicts(var, val, assignment, constraints, neighbors):
    "Return the number of conflicts var=val has with other variables."
    # Subclasses may implement this more efficiently
    def conflict(var2):
        val2 = assignment.get(var2, None)
        return val2 != None and not constraints(var, val, var2, val2)
    return len(list(filter(conflict, neighbors[var])))

def argmin_random_tie(seq, fn):
    """Return an element with lowest fn(seq[i]) score; break ties at random.
    Thus, for all s,f: argmin_random_tie(s, f) in argmin_list(s, f)"""
    best_score = fn(seq[0]); n = 0
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score; n = 1
        elif x_score == best_score:
            n += 1
            if random.randrange(n) == 0:
                    best = x
    return best

## Required Functions

See method bodies for explanations.

### Schedule Function

In [3]:
def schedule(classes, times, rooms, max_steps):
    """
    Args:
        - classes: list of all class names, like 'CS410'
        - times: list of all start times, like '10 am' and ' 1 pm'
        - rooms: list of all rooms, like 'CSB 325'
        - max_steps: maximum number of assignments to try
    
    Returns:
        - assignments: dictionary of values assigned to variables, like
          {'CS410': ('CSB 425', '10 am'), ...}
        - steps: actual number of assignments tested before solution found
          assignments and steps will each be None if solution was not found.
    """
    
    # Variables:
    # Each class in classes is a variable which we are trying to solve given
    # contraints.
    
    # Domain:
    # Each variable maps to a tuple of (room, time). No variable is unique in
    # anyway, so the domain is the same for all. The domain is a combination of
    # room and time. We build the domain as such:
    domain = []
    for room in rooms:
        for time in times:
            domain.append((room, time))
    # Then build map for every variable to this domain.
    domains = {var: domain for var in classes}

    # Again, nothing special about any class, so the neighbors to check for 
    # constraints are just everyone else.
    neighbors = {var: [v for v in classes if v != var] for var in classes}
    
    # Once we have defined the problem as above, we can use min_conflicts to
    # solve it.
    return min_conflicts(classes, domains, constraints_ok, neighbors, max_steps)

### Constraints Function

In [4]:
def constraints_ok(class_name_1, value_1, class_name_2, value_2):
    """
    Args:
        - class_name_1: as above, like 'CS410'
        - value_1: tuple containing room and time.
        - class_name_2: a second class name
        - value_2: another tuple containing a room and time
    
    Returns:
        - result: True of the assignment of value_1 to class_name 1 and value_2
          to class_name 2 does not violate any constraints.  False otherwise.

    Constraints:
        - Two classes cannot meet in the same room at the same time.
        - Classes with the same first digit cannot meet at the same time,
          because students might take a subset of these in one semester. There
          is one exception to this rule. CS163 and CS164 can meet at the same
          time.
    """
    # Renaming variables for easier reading.
    c1, c2 = class_name_1, class_name_2
    r1, t1 = value_1
    r2, t2 = value_2
    
    # First, we check if its the same variable. This will never be true
    # because we specified neighbors. It seems like the Queen code put it
    # here, so I will too. It will filter a poor problem definition.
    if c1==c2:
        return False
    
    # Second, we check if the assignments are trying to use the same room at
    # the same time
    if r1==r2 and t1==t2: # constraint 1
        return False # they are
    
    # Third, we check if they have the same hundred level class and are at the
    # same time.
    elif c1[2] == c2[2] and t1==t2: # constraint 2
        # If they are, we need to see if they are the exception.
        return (c1 == 'CS163' and c2=='CS164') or (c1 == 'CS164' and c2=='CS163')
    
    # Finally, it passed all the constraints, so return true.
    else:
        return True

### Display Function

In [5]:
def display(assignments, rooms, times):
    """
    Args:
        - assignments: returned from call to your schedule function
        - rooms: list of all rooms as above
        - times: list of all times as above pass
    """
    # First, we check to see if there is anything to print. If not, no
    # solution was found, and we print that.
    if assignments == None:
        print("No solution found!")
        return

    # Prep:
    # We need to print classes in a table: times by rooms. Printing moves
    # vertical, so we need to have a dictionary for times -> rooms -> class.
    # Then we can iterate times, print for each room the class. Assignments
    # comes in the format of class -> (room, time), so I reshape that:
    printMap = {} # times -> rooms -> classes
    timeSpace = len(times[0])
    for className in assignments:
        room, time = assignments[className]
        if time not in printMap:
            printMap[time] = {}
        printMap[time][room] = className
    
    # Print:
    # Printing is pretty standard stuff. Print the header (the rooms). Then print each row
    # Each row starts with the time, and then prints the classes.
    timeSpace = len(times[-1])
    print(' '*timeSpace, end='')
    for room in rooms:
        print('{:>9}'.format(room), end='')
    print()
    print('-'*(timeSpace+9*len(rooms)))
    for time in times:
        print(time, end='')
        for room in rooms:
            toPrint = printMap.get(time, {}).get(room, '')
            print('{:>9}'.format(toPrint), end='')
        print()

## Results

First, we need to define the classes, times, and rooms:

In [6]:
classes = ['CS160', 'CS163', 'CS164',
           'CS220', 'CS270', 'CS253',
           'CS320', 'CS314', 'CS356', 'CS370',
           'CS410', 'CS414', 'CS420', 'CS430', 'CS440', 'CS445', 'CS453', 'CS464',
           'CS510', 'CS514', 'CS535', 'CS540', 'CS545']

times = [' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']

rooms = ['CSB 130', 'CSB 325', 'CSB 425']

Here is an example of solving the problem:

In [7]:
max_steps = 100
assignments, steps = schedule(classes, times, rooms, max_steps)
print('Took', steps, 'steps')
print(assignments)

Took 0 steps
{'CS160': ('CSB 425', ' 2 pm'), 'CS163': ('CSB 325', ' 9 am'), 'CS164': ('CSB 130', '11 am'), 'CS220': ('CSB 425', ' 3 pm'), 'CS270': ('CSB 425', '10 am'), 'CS253': ('CSB 130', ' 4 pm'), 'CS320': ('CSB 325', '12 pm'), 'CS314': ('CSB 325', ' 3 pm'), 'CS356': ('CSB 425', ' 1 pm'), 'CS370': ('CSB 425', ' 4 pm'), 'CS410': ('CSB 325', '10 am'), 'CS414': ('CSB 425', ' 9 am'), 'CS420': ('CSB 425', '11 am'), 'CS430': ('CSB 130', ' 2 pm'), 'CS440': ('CSB 325', ' 1 pm'), 'CS445': ('CSB 130', '12 pm'), 'CS453': ('CSB 130', ' 3 pm'), 'CS464': ('CSB 325', ' 4 pm'), 'CS510': ('CSB 325', ' 2 pm'), 'CS514': ('CSB 130', ' 1 pm'), 'CS535': ('CSB 130', ' 9 am'), 'CS540': ('CSB 425', '12 pm'), 'CS545': ('CSB 325', '11 am')}


Here is a more readable interpretation:

In [8]:
display(assignments, rooms, times)

       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am    CS535    CS163    CS414
10 am             CS410    CS270
11 am    CS164    CS545    CS420
12 pm    CS445    CS320    CS540
 1 pm    CS514    CS440    CS356
 2 pm    CS430    CS510    CS160
 3 pm    CS453    CS314    CS220
 4 pm    CS253    CS464    CS370


Here is the algorithm being run 8 times.

In [9]:
for _ in range(8):
    max_steps = 100
    assignments, steps = schedule(classes, times, rooms, max_steps)
    print('Took', steps, 'steps')
    display(assignments, rooms, times)
    print("*"*50)

Took 0 steps
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am             CS220    CS440
10 am    CS464    CS163    CS535
11 am    CS445    CS270    CS545
12 pm    CS510    CS370    CS410
 1 pm    CS430    CS164    CS314
 2 pm    CS420    CS160    CS356
 3 pm    CS453    CS320    CS514
 4 pm    CS253    CS414    CS540
**************************************************
Took 3 steps
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am    CS370    CS540    CS445
10 am    CS320    CS453    CS220
11 am    CS164    CS464    CS510
12 pm    CS270    CS535    CS414
 1 pm    CS163    CS545    CS440
 2 pm    CS160    CS430         
 3 pm    CS356    CS253    CS410
 4 pm    CS420    CS514    CS314
**************************************************
Took 0 steps
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am    CS440    CS160    CS535
10 am             CS163    CS464
11 am    CS514    CS320    CS430
12 pm    CS370    CS410    CS164
 

## Discussion

It solves this problem really fast. Most of the time, it seems to get it on the first try. Also, it doesn't take any time to run since it hardly has to iterate. 

Looking at the 8 generated solutions, we can see that the solutions satisfy the constraints. In the following example, it even includes the CS163/CS164 being at the same time exception.

    Took 0 steps
           CSB 130  CSB 325  CSB 425
    --------------------------------
     9 am    CS545    CS314    CS430
    10 am    CS464    CS370    CS160
    11 am    CS420    CS514         
    12 pm    CS510    CS440    CS253
     1 pm    CS270    CS453    CS356
     2 pm    CS445    CS535    CS220
     3 pm    CS163    CS164    CS410
     4 pm    CS414    CS540    CS320

Also, we can see from the 8 generated solutions that the solutions are usually different. There are quite a few that it can find. This is possibly why it returns values so quickly.

## Grading

Download [A6grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A6grader.tar) and extract `A6grader.py` from it.  Grader will be available soon.

In [10]:
%run -i A6grader.py

ERROR:root:File `'A6grader.py'` not found.


# Extra Credit

In [11]:
def preferenceViolations(c, v):
    """
    This method counts the number of preferences that are violated. This
    might have hte issue of preferences not being weighted properly, but 
    that's an easy fix and ok. I don't care, so I'll ignore it.
    Especially since the preferences are mutually exclusive (meaning this
    method will never return more than 1).
    
    Preferences:
        - prefer schedules that do not schedule classes at 9 am, 12 pm or 4 pm.
        - prefer schedules with CS163 and CS164 meeting at 1 pm or 2 pm.
    """
    violations=0
    r,t = v
    if t==' 9 am' or t=='12 pm' or t==' 4 pm': # preference 1
        violations += 1
    if c=='CS163' or c=='CS164': # preference 2 applies
        if t!=' 1 pm' and t!=' 2 pm': # preference 2
            violations += 1
    return violations

def countConflict(assignments):
    """
    This will iterate over every pair of assignments and check for a
    preference violation. It counts and returns how many there are.
    Also, note that it will double count every pair, so really, it's
    double the number of violations, but we only care about the
    magnitude, so that's not an issue. Plus, the number value
    is arbuitrary anyways.
    """
    count=0
    for c in assignments:
        count += preferenceViolations(c, assignments[c])
    return count

def getBestAssignments(classes, times, rooms, max_steps, tries=100, getCount=False):
    """
    This function is meant to have the same signature as schedule. For
    `tries` tries, it will generate a solution. It will evaluate how many
    assignments violate a preference. It counts them. It values the best
    configuration as the one with the smallest number of preference
    violations.
    """
    bestAssignments = None
    for _ in range(tries):
        assignments, steps = schedule(classes, times, rooms, max_steps)
        count = countConflict(assignments)
        if bestAssignments is None or bestCount > count:
            bestCount = count
            bestAssignments = assignments
            bestSteps = steps
    if getCount:
        return bestAssignments, bestSteps, count
    else:
        return bestAssignments, bestSteps

In [12]:
classes = ['CS160', 'CS163', 'CS164',
           'CS220', 'CS270', 'CS253',
           'CS320', 'CS314', 'CS356', 'CS370',
           'CS410', 'CS414', 'CS420', 'CS430', 'CS440', 'CS445', 'CS453', 'CS464',
           'CS510', 'CS514', 'CS535', 'CS540', 'CS545']

times = [' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']

rooms = ['CSB 130', 'CSB 325', 'CSB 425']

In [13]:
max_steps = 100
assignments, steps, count = getBestAssignments(classes, times, rooms, max_steps, getCount=True)
print("*"*50)
print('Took', steps, 'steps')
print('Preference count:', count)
display(assignments, rooms, times)

**************************************************
Took 0 steps
Preference count: 10
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am             CS510    CS420
10 am    CS545    CS410    CS253
11 am    CS464    CS320    CS220
12 pm    CS445    CS356    CS514
 1 pm    CS414    CS164    CS163
 2 pm    CS270    CS453    CS314
 3 pm    CS535    CS430    CS370
 4 pm    CS440    CS160    CS540


In [14]:
for _ in range(8):
    max_steps = 100
    assignments, steps, count = getBestAssignments(classes, times, rooms, max_steps, getCount=True)
    print("*"*50)
    print('Took', steps, 'steps')
    print('Preference count:', count)
    display(assignments, rooms, times)

**************************************************
Took 0 steps
Preference count: 10
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am    CS464    CS545    CS356
10 am    CS160    CS535    CS414
11 am    CS540    CS440    CS314
12 pm    CS510    CS420         
 1 pm    CS163    CS410    CS253
 2 pm    CS430    CS164    CS220
 3 pm    CS445    CS320    CS270
 4 pm    CS370    CS453    CS514
**************************************************
Took 0 steps
Preference count: 10
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am             CS514    CS430
10 am    CS370    CS420    CS253
11 am    CS270    CS160    CS464
12 pm    CS314    CS410    CS545
 1 pm    CS510    CS163    CS445
 2 pm    CS440    CS220    CS164
 3 pm    CS320    CS414    CS540
 4 pm    CS453    CS356    CS535
**************************************************
Took 0 steps
Preference count: 11
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am    CS440     

This is the same analysis as before, except now we can see that the algorithm favors our preferences (despite violating them). The gap always shows up at 9, 12 or 4. CS 163/4 always shows up at 1 or 2 pm. It's clear that the preferences are generally being met.

The algorithm reports 11 preference violations. These violations are forced since there isn't enough room (8 classes during 9, 12, and 4). However, if we remove 11 classes, then we can expect the algorithm to find the solution meeting the preferences completely.

In [15]:
classes = ['CS160', 'CS163', # removed 1
           'CS220', 'CS270', # removed 1
           'CS320', 'CS314', # removed 2
           'CS410', 'CS414', 'CS420', # removed 5
           'CS510', 'CS514', 'CS535', # removed 2
          ] # removed 11 total

times = [' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']

rooms = ['CSB 130', 'CSB 325', 'CSB 425']

In [16]:
max_steps = 100
assignments, steps, count = getBestAssignments(classes, times, rooms, max_steps, getCount=True)
print("*"*50)
print('Took', steps, 'steps')
print('Preference count:', count)
display(assignments, rooms, times)

**************************************************
Took 0 steps
Preference count: 6
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am                           
10 am    CS160    CS535         
11 am    CS270             CS314
12 pm    CS420             CS514
 1 pm    CS320             CS163
 2 pm    CS410                  
 3 pm    CS510    CS414    CS220
 4 pm                           


Preferences are kind of being met, but the issue with the algorithm is being highlighted. Since it essentially randomly generates solutions until the constraints are met, it doesn't necessarily find the best solution, as defined by our preferences. This is why it finds a solution, but can't eliminate our preferences. It's essentially trying to guess the solution. (a terrible search method).

Maybe it needs to iterate more?

In [17]:
tries = 10000
assignments, steps, count = getBestAssignments(classes, times, rooms, max_steps, tries=tries, getCount=True)
print("*"*50)
print('Took', steps, 'steps')
print('Preference count:', count)
display(assignments, rooms, times)

**************************************************
Took 0 steps
Preference count: 5
       CSB 130  CSB 325  CSB 425
--------------------------------
 9 am                           
10 am             CS270    CS535
11 am    CS160    CS320    CS514
12 pm                           
 1 pm    CS314             CS420
 2 pm    CS163    CS220         
 3 pm    CS510             CS410
 4 pm             CS414         


Nope.

Let's redefine the problem to make the preferences part of the constraints. Without modifying algorithms, the easiest way to do this is to eliminate time 9, 12 and 4 time options. Finally, we apply the preferences, and the second preferences will likely be met because it is easy.

In [18]:
times = ['10 am',
         '11 am',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm']
tries = 10000
assignments, steps, count = getBestAssignments(classes, times, rooms, max_steps, tries=tries, getCount=True)
print("*"*50)
print('Took', steps, 'steps')
print('Preference count:', count)
display(assignments, rooms, times)

**************************************************
Took 0 steps
Preference count: 0
       CSB 130  CSB 325  CSB 425
--------------------------------
10 am    CS414    CS535    CS220
11 am                      CS510
 1 pm    CS420    CS163    CS314
 2 pm    CS410    CS160    CS270
 3 pm    CS514    CS320         


This is consistent with my argument: The ideal solution is a preference violation of 0, but my algorithm above couldn't find this. Randomly generating solutions is a terrible way of optimization.